In [1]:
import pandas as pd
import numpy as np

In [2]:
dados = pd.read_csv('DB_Teste.csv', sep=';')
dados.head()

,Cliente,ID,Tipo,Data da Venda,Categoria,Vendedor,Regional,Duração do Contrato (Meses),Equipe,Valor,Unnamed: 10
0,Cliente 1,2017-0022,Serviços,02/01/2018,Novo Logo,Vendedor 1,Brasil,1,Time 1,"R$ 24.800,00",NaN
1,Cliente 10,2018-0088,Serviços,14/02/2018,Novo Logo,Vendedor 1,Brasil,6,Time 1,"R$ 302.720,00",NaN
2,Cliente 10,2018-0104,Serviços,13/03/2018,Upselling,Vendedor 1,Brasil,1,Time 1,"R$ 51.894,86",NaN
3,Cliente 10,2018-0168,Serviços,30/04/2018,Upselling,Vendedor 1,Brasil,2,Time 1,"R$ 51.894,86",NaN
4,Cliente 10,2018-0169,Serviços,30/04/2018,Upselling,Vendedor 1,Brasil,2,Time 1,"R$ 51.894,86",NaN


In [3]:
dados.drop(columns={'Unnamed: 10'}, inplace=True)

In [4]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853 entries, 0 to 852
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Cliente                      853 non-null    object
 1   ID                           853 non-null    object
 2   Tipo                         853 non-null    object
 3   Data da Venda                853 non-null    object
 4   Categoria                    853 non-null    object
 5   Vendedor                     853 non-null    object
 6   Regional                     853 non-null    object
 7   Duração do Contrato (Meses)  853 non-null    int64 
 8   Equipe                       853 non-null    object
 9   Valor                        853 non-null    object
dtypes: int64(1), object(9)
memory usage: 66.8+ KB


# 1) Construa uma tabela auxiliar que sumarize o valor vendido por cada vendedor, ordenando do maior para o menor.

In [5]:
dados['Valor'] = dados['Valor'].apply(lambda x: float(x.replace('R$', '').replace('.', '').replace(',', '.')))

In [6]:
import locale
def transformacao_valor(x):
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
    x = locale.currency(x, grouping=True, symbol='R$')
    return x

In [7]:
tabela_vendas_por_vendedor = dados.groupby(by='Vendedor', as_index=False)[['Valor']].sum().sort_values(by='Valor', ascending=False)
tabela_vendas_por_vendedor['Valor'] = tabela_vendas_por_vendedor['Valor'].apply(lambda x: transformacao_valor(x))
tabela_vendas_por_vendedor.reset_index(drop=True, inplace=True)
tabela_vendas_por_vendedor

,Vendedor,Valor
0,Vendedor 3,"R$ 166.220.477,72"
1,Vendedor 7,"R$ 57.799.181,73"
2,Vendedor 10,"R$ 52.212.055,49"
3,Vendedor 8,"R$ 47.274.510,01"
4,Vendedor 1,"R$ 30.929.480,27"
5,Vendedor 9,"R$ 26.374.558,54"
6,Vendedor 13,"R$ 24.837.304,87"
7,Vendedor 18,"R$ 22.662.296,25"
8,Vendedor 29,"R$ 19.076.866,26"
9,Vendedor 4,"R$ 17.346.823,03"


# 2) Imprima e identifica qual foi o cliente responsável pela venda com maior valor e com menor valor.

In [8]:
maior_valor = max(dados['Valor'])
menor_valor = min(dados['Valor'])

cliente_maior_valor = dados.where(dados['Valor'] == maior_valor)
cliente_menor_valor = dados.where(dados['Valor'] == menor_valor)

cliente_maior_valor.dropna(inplace=True)
cliente_menor_valor.dropna(inplace=True)

cliente_maior_valor = cliente_maior_valor.reset_index(drop=True)
cliente_menor_valor = cliente_menor_valor.reset_index(drop=True)

cliente_maior_valor['Valor'] = cliente_maior_valor['Valor'].apply(lambda x: transformacao_valor(x))
cliente_menor_valor['Valor'] = cliente_menor_valor['Valor'].apply(lambda x: transformacao_valor(x))

print('O cliente responsável pela venda com maior valor foi o {}, com o valor de {}'.format(cliente_maior_valor['Cliente'][0], cliente_maior_valor['Valor'][0]))
print('O cliente responsável pela venda com menor valor foi o {}, com o valor de {}'.format(cliente_menor_valor['Cliente'][0], cliente_menor_valor['Valor'][0]))

O cliente responsável pela venda com maior valor foi o Cliente 89, com o valor de R$ 24.970.500,00
O cliente responsável pela venda com menor valor foi o Cliente 120, com o valor de R$ 1.250,00


# 3) Imprima valor médio por Tipo de venda (Serviços, Licenciamento, Produtos).

In [9]:
tabela_valor_medio_tipo_venda = dados.groupby(by='Tipo', as_index=False)[['Valor']].mean()
tabela_valor_medio_tipo_venda['Valor'] = tabela_valor_medio_tipo_venda['Valor'].apply(lambda x: transformacao_valor(x))

for i in range(0, len(tabela_valor_medio_tipo_venda)):
    print('Tipo: {} / Valor médio: {}'.format(tabela_valor_medio_tipo_venda['Tipo'][i], tabela_valor_medio_tipo_venda['Valor'][i]))

Tipo: Licenciamento / Valor médio: R$ 983.448,77
Tipo: Produtos / Valor médio: R$ 893.409,19
Tipo: Serviços / Valor médio: R$ 314.567,94


# 4) Imprima o número de vendas realizada por cliente.

In [10]:
tabela_num_vendas_cliente = dados.groupby(by='Cliente', as_index=False)[['ID']].count()
tabela_num_vendas_cliente.rename(columns={'ID':'Numero de Vendas'}, inplace=True)

for i in range(0, len(tabela_num_vendas_cliente)):
    print('Cliente: {} - Número de Vendas: {}'.format(tabela_num_vendas_cliente['Cliente'][i], tabela_num_vendas_cliente['Numero de Vendas'][i]))

Cliente: Cliente 1 - Número de Vendas: 1
Cliente: Cliente 10 - Número de Vendas: 43
Cliente: Cliente 100 - Número de Vendas: 1
Cliente: Cliente 101 - Número de Vendas: 2
Cliente: Cliente 102 - Número de Vendas: 1
Cliente: Cliente 103 - Número de Vendas: 1
Cliente: Cliente 104 - Número de Vendas: 1
Cliente: Cliente 105 - Número de Vendas: 1
Cliente: Cliente 106 - Número de Vendas: 1
Cliente: Cliente 107 - Número de Vendas: 1
Cliente: Cliente 108 - Número de Vendas: 2
Cliente: Cliente 109 - Número de Vendas: 5
Cliente: Cliente 11 - Número de Vendas: 1
Cliente: Cliente 110 - Número de Vendas: 1
Cliente: Cliente 111 - Número de Vendas: 2
Cliente: Cliente 112 - Número de Vendas: 1
Cliente: Cliente 113 - Número de Vendas: 1
Cliente: Cliente 114 - Número de Vendas: 1
Cliente: Cliente 115 - Número de Vendas: 1
Cliente: Cliente 116 - Número de Vendas: 3
Cliente: Cliente 117 - Número de Vendas: 1
Cliente: Cliente 118 - Número de Vendas: 2
Cliente: Cliente 119 - Número de Vendas: 1
Cliente: Clien